# <center>  ML INTERNSHIP - HACKEREARTH CHALLENGE

**1. importing necssary libraries**

In [53]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import nltk
from nltk.corpus import stopwords
%matplotlib inline

In [26]:
train_df = pd.read_csv('hm_train.csv')

In [27]:
train_df.head()

,hmid,reflection_period,cleaned_hm,num_sentence,predicted_category
0,27673,24h,I went on a successful date with someone I fel...,1,affection
1,27674,24h,I was happy when my son got 90% marks in his e...,1,affection
2,27675,24h,I went to the gym this morning and did yoga.,1,exercise
3,27676,24h,We had a serious talk with some friends of our...,2,bonding
4,27677,24h,I went with grandchildren to butterfly display...,1,affection


In [28]:
train_df.describe()

,hmid,num_sentence
count,60321.000000,60321.000000
mean,57996.929510,1.355946
std,17501.024854,1.308160
min,27673.000000,1.000000
25%,42845.000000,1.000000
50%,58001.000000,1.000000
75%,73160.000000,1.000000
max,88303.000000,58.000000


In [29]:
train_df.describe(include=['object'])

,reflection_period,cleaned_hm,predicted_category
count,60321,60321,60321
unique,2,58454,7
top,24h,I WENT TO MOVIE,affection
freq,30455,76,20880


In [30]:
train_df.info() # clearly no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60321 entries, 0 to 60320
Data columns (total 5 columns):
hmid                  60321 non-null int64
reflection_period     60321 non-null object
cleaned_hm            60321 non-null object
num_sentence          60321 non-null int64
predicted_category    60321 non-null object
dtypes: int64(2), object(3)
memory usage: 2.3+ MB


In [31]:
train_df['cleaned_hm'].describe() # unique != count there can be repeated messages

count               60321
unique              58454
top       I WENT TO MOVIE
freq                   76
Name: cleaned_hm, dtype: object

In [32]:
train_df.groupby(['predicted_category'])['cleaned_hm'].describe()

,count,unique,top,freq
predicted_category,,,,
achievement,20274,19832,I got a new job.,19
affection,20880,20384,My boyfriend took me on a romantic date.,18
bonding,6561,6335,The offsite with colleagues was great fun. We ...,38
enjoy_the_moment,6508,6230,Happiness is a mental or emotional state of we...,24
exercise,729,684,I WENT TO YOGA,14
leisure,4242,3870,I WENT TO MOVIE,76
nature,1127,1121,Summer is very bad weather for plants and tree...,2


In [33]:
train_df.groupby('predicted_category')['num_sentence'].describe()

,count,mean,std,min,25%,50%,75%,max
predicted_category,,,,,,,,
achievement,20274.0,1.233945,0.791342,1.0,1.0,1.0,1.0,21.0
affection,20880.0,1.474713,1.686058,1.0,1.0,1.0,1.0,58.0
bonding,6561.0,1.363664,1.197323,1.0,1.0,1.0,1.0,29.0
enjoy_the_moment,6508.0,1.529502,1.671654,1.0,1.0,1.0,1.0,46.0
exercise,729.0,1.144033,0.471548,1.0,1.0,1.0,1.0,5.0
leisure,4242.0,1.113390,0.463845,1.0,1.0,1.0,1.0,10.0
nature,1127.0,1.353150,1.129865,1.0,1.0,1.0,1.0,18.0


In [69]:
train_df['len_of_text'] = train_df['cleaned_hm'].apply(len)

# 2.text preprocessing

In [58]:
# creating text pre-process
def text_pre_process(text):
    """
        1.remove punctuation
        2.remove stop words
        3.lower case all words
        return list of clean text words
    """
    nonpunc = ''.join([c for c in text if c not in string.punctuation])
    
    return [word.lower() for word in nonpunc.split() if word not in stopwords.words('english')]

In [65]:
# now apply to text column
train_df['preprocessed_text'] = train_df['cleaned_hm'].apply(text_pre_process)

In [67]:
train_df['preprocessed_text'].head()

0    [i, went, successful, date, someone, i, felt, ...
1         [i, happy, son, got, 90, marks, examination]
2                        [i, went, gym, morning, yoga]
3    [we, serious, talk, friends, flaky, lately, th...
4    [i, went, grandchildren, butterfly, display, c...
Name: preprocessed_text, dtype: object

In [71]:
# using count vectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [72]:
bow_transformer = CountVectorizer(analyzer=text_pre_process).fit(train_df['cleaned_hm'] )

In [73]:
print(len(bow_transformer.vocabulary_))

23122


In [78]:
bow1 = bow_transformer.transform([text1,] )

In [83]:
print(bow1)

  (0, 5285)	1
  (0, 7912)	1
  (0, 10284)	1
  (0, 12475)	1
  (0, 12702)	1
  (0, 20935)	1


In [100]:
print(bow_transformer.get_feature_names()[5285])

crispy


In [108]:
bag_of_words = bow_transformer.transform(train_df['cleaned_hm'])

# tf-idf vectorizer

In [109]:
from sklearn.feature_extraction.text import TfidfTransformer

In [110]:
 tfidf_transformer = TfidfTransformer().fit(bag_of_words)

In [117]:
print(tfidf_transformer.transform(bow1))

  (0, 20935)	0.5633772314809447
  (0, 12702)	0.5203631242766186
  (0, 12475)	0.27071541467174004
  (0, 10284)	0.07543265064167214
  (0, 7912)	0.23565352659565988
  (0, 5285)	0.526610680504346


In [118]:
text_tfidf = tfidf_transformer.transform(bag_of_words)

In [119]:
from sklearn.naive_bayes import MultinomialNB

In [120]:
model = MultinomialNB()

In [121]:
model.fit(text_tfidf, train_df['predicted_category'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [122]:
model.predict(bow1)

array(['achievement'], dtype='<U16')

In [127]:
print(train_df[train_df['cleaned_hm']==text1]['predicted_category'])

145    achievement
Name: predicted_category, dtype: object


In [128]:
train_df['cleaned_hm'].describe() # unique != count there can be repeated messages

count               60321
unique              58454
top       I WENT TO MOVIE
freq                   76
Name: cleaned_hm, dtype: object